# Jaynes Cumming Model LMAO... 23yo bro

In [170]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from scipy.signal import find_peaks

# Sayak please look at the comments for the explanation of each step:

# =============================================================================
# Define Experimental Parameters (in SI units)
# =============================================================================
# Experimental values (in Hz)
wc = 2 * np.pi * 5e9   # cavity frequency (5 GHz)
wa = 2 * np.pi * 5e9   # atomic transition frequency (5 GHz)
g  = 2 * np.pi * 1e7   # coupling strength (10 MHz)

# This values were defined by ChatGPT based on similarity with superconducting qubits' values.
# wc and wa are set at resonance, we might change this, but I think Florian wanted this asw.

# =============================================================================
# Define Unitless (Dimensionless) Parameters
# =============================================================================
# We factor out ℏ*wc so that the unitless Hamiltonian becomes:
#    H' = a†a + 0.5*(wa/wc)*σ_z + (g/wc)*(a σ⁺ + a† σ⁻)
g_eff = g / wc       # unitless coupling
delta_eff = (wa - wc) / wc  # unitless detuning, this should be zero since wa == wc, set at resonance at the beginning -^

# This follows exactly our initial code.

# =============================================================================
# Set Hilbert Space Dimensions and Define Operators
# =============================================================================
N = 75  # number of cavity Fock states to include, we might increase this but it's basically the same.

# Define operators on the full (cavity ⊗ atom) space.
# Note: In our tensor-product, the cavity Hilbert space is first, and the atom second. This follows qutip documentation.
a      = tensor(destroy(N), qeye(2))   # cavity annihilation operator
adag   = a.dag()                       # cavity creation operator
num_cav = adag * a                     # cavity photon number operator

# Two-level system (atom) operators (QuTiP’s sigmam() and sigmap() use basis |e>,|g> with sigmaz = diag(1, -1))
sm = tensor(qeye(N), sigmam())  # atomic lowering operator
sp = tensor(qeye(N), sigmap())  # atomic raising operator
sz = tensor(qeye(N), sigmaz())  # atomic inversion

# =============================================================================
# Build the Unitary (Time-Independent) Hamiltonian
# =============================================================================
# The standard JC Hamiltonian (with ℏ factored out) is:
#   H = ℏ wc (a†a) + (ℏ wa/2) σ_z + ℏ g (a σ⁺ + a† σ⁻)
# Dividing through by ℏ wc, we get the dimensionless Hamiltonian:
#
#   H' = a†a + 0.5*(wa/wc)*σ_z + (g/wc)*(a σ⁺ + a† σ⁻)
H = num_cav + 0.5 * (wa / wc) * sz + g_eff * (a * sp + adag * sm)

# Again as our initial code.

# =============================================================================
# Time Evolution for a Single Initial State
# =============================================================================
# For example, let us simulate the case where the system has one total excitation.
# In the JC model the total excitation number is N_ex = n_cavity + (σ_z+1)/2.
# For N_ex = 1 the relevant two-level subspace is spanned by:
#      |n=0, e>   and   |n=1, g>
# We choose the initial state to be |0, e> (i.e. cavity vacuum and atom excited).
psi0 = tensor(fock(N, 0), basis(2, 0))

# Choose a time grid in the unitless time variable (wc*t).
# For |0,e> the Rabi frequency in these units is Ω = 2*g_eff (since √1 = 1),
# so the Rabi period is T_R = 2π/(2*g_eff) = π/g_eff.
# With g_eff = g/wc ≈ 1e7/5e9 = 0.002, T_R ~ π/0.002 ≈ 1570.
t_max = 5000
n_steps = 5000
tlist = np.linspace(0, t_max, n_steps) # We chose even higher values for the plot scale, might increase.

# Define the projection onto the excited state (atom) operator:
excited_proj = tensor(qeye(N), basis(2, 0) * basis(2, 0).dag())

# Solve the time-dependent Schrödinger equation (no collapse operators).
result = mesolve(H, psi0, tlist, [], [num_cav, excited_proj])

# Plot the cavity photon number and atomic excitation probability vs. time.
plt.figure(figsize=(8, 5))
plt.plot(tlist, result.expect[0], label="Cavity Photon Number")
plt.plot(tlist, result.expect[1], label="Atom Excitation Probability")
plt.xlabel("Dimensionless Time (ω_c t)")
plt.ylabel("Expectation Value")
plt.title("JC Dynamics: Cavity & Atom Observables")
plt.legend()
plt.tight_layout()
plt.show()

# =============================================================================
# Verifying the Simulation: Extracting the Rabi Frequencies
# =============================================================================
# In the JC model, if the total number of excitations is n (n>=1), then if we
# start with the state |n-1, e> (i.e. n-1 photons in the cavity and the atom excited),
# the evolution is confined to the two-level subspace {|n-1, e>, |n, g>}.
# In that manifold the probability oscillates with frequency:
#
#      Ω_theory = 2 * g_eff * √(n)
#
# We now loop over several manifolds (n = 1, 2, ..., n_max) to simulate the evolution,
# extract the oscillation period from the atomic excited-state population, and then compare.
n_manifolds = np.arange(1, 11)  # simulate for n = 1,2,...,10 (total excitation numbers)
omega_exp_list = []  # will hold frequencies extracted from simulation (in units of ω_c)
omega_th_list = []   # theoretical frequencies

for n in n_manifolds:
    # For a total excitation number n, prepare the initial state |n-1, e>
    psi0_n = tensor(fock(N, n-1), basis(2, 0))

    # This should explain why we were getting matching frequencies at this rate. I will tell you more about it.
    
    # For each manifold the Rabi frequency is Ω_theory = 2*g_eff*√(n),
    # and the Rabi period T_R = 2π / Ω_theory.
    Omega_theory = 2 * g_eff * np.sqrt(n)
    T_R = 2 * np.pi / Omega_theory
    omega_th_list.append(Omega_theory)
    
    # Set a time grid that spans a few Rabi periods (say, 3 periods)
    t_max_n = 3 * T_R
    n_steps_n = 1000  # use sufficient resolution for the shorter time scale
    tlist_n = np.linspace(0, t_max_n, n_steps_n)
    
    # Solve the dynamics for the state |n-1, e>
    result_n = mesolve(H, psi0_n, tlist_n, [], [excited_proj])
    Pe = result_n.expect[0]  # atomic excitation probability vs. time
    
    # Use scipy.signal.find_peaks to locate the peaks (maxima) of the oscillation.
    # (Since the oscillation is nearly sinusoidal, the first peak should be at t = 0.)
    peaks, _ = find_peaks(Pe, height=0.5)
    
    # We need at least two peaks to extract a period.
    if len(peaks) >= 2:
        # Compute the period from differences in the time array at the peak indices.
        T_meas = np.mean(np.diff(tlist_n[peaks]))
        # Extract the oscillation frequency (in these unitless units)
        Omega_meas = 2 * np.pi / T_meas
    else:
        Omega_meas = np.nan  # not enough peaks to determine period
    
    omega_exp_list.append(Omega_meas)

# Convert lists to arrays for plotting.
omega_exp_list = np.array(omega_exp_list)
omega_th_list = np.array(omega_th_list)

# Scatter plot comparing simulated (experimental) vs. theoretical Rabi frequencies.
plt.figure(figsize=(6, 5))
plt.scatter(n_manifolds, omega_exp_list, color='b', label="Simulated Ω")
plt.plot(n_manifolds, omega_th_list, 'r--', label="Theoretical Ω = 2 g_eff √n")
plt.xlabel("Total Excitation Number n")
plt.ylabel("Rabi Frequency (in units of ω_c)")
plt.title("Comparison of Rabi Frequencies")
plt.legend()
plt.tight_layout()
plt.show()

In [132]:
# Sample of simulation using Qutip

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from qutip import (basis, destroy, mesolve, qeye,tensor, SESolver, sigmaz, sigmam, sigmap, fock)
from scipy.signal import find_peaks

matplotlib.use('TkAgg')

# System parameters
wc = 1.0 * 2 * np.pi  # cavity frequency
wa = 1.0 * 2 * np.pi  # atom frequency 
g = 0.5 * 2 * np.pi # coupling strength

# Hilbert space dimension for the cavity
n_cavity = 75

# Operators
sigma_minus = tensor(qeye(n_cavity), destroy(2)) # atomic lowering operator
sigma_plus = sigma_minus.dag()   # atomic raising operator
sigma_z = tensor(qeye(n_cavity), sigmaz())     # atomic z operator
a = tensor(destroy(n_cavity), qeye(2))   # cavity annihilation operator

# Times for which the state should be evaluated
times = np.linspace(0, 25, 50000)

tau = wc * times


# Hamiltonian with rotating wave approximation
H_theoretical =  a.dag() * a + 0.5 * (wa/wc) * sigma_minus*sigma_plus +  (g/wc) * (a.dag() * sigma_minus + a * sigma_plus)

H_experimental = wc * H_theoretical


# Initial state: superposition of state
psi0 = tensor(fock(n_cavity, 1), basis(2, 1)) + tensor(fock(n_cavity, 2), basis(2, 0))


# Solve the Schrodinger equation
result = mesolve(H_theoretical, psi0, tau, c_ops=[], e_ops=[a.dag() * a, sigma_plus * sigma_minus])
#result_alt =  mesolve(H_experimental, psi0, times, c_ops=[], e_ops=[a.dag() * a, sigma_plus * sigma_minus])
#solver = SESolver(H_theoretical)
#result = solver.run(psi0, tau, e_ops = [a.dag() * a, sigma_plus * sigma_minus])

n_c = result.expect[0]
n_a = result.expect[1]

fig, axes = plt.subplots(1, 1, figsize=(10, 6))#
axes.plot(result.times, n_c, label="Cavity")
axes.plot(result.times, n_a, label="Atom Excitation")
axes.grid(True)
axes.legend(loc='best')
axes.set_xlabel("Time (arb. units)")
axes.set_xlim(0,20)
axes.set_ylabel("Photon number")
axes.set_title("Initial States: (2,g) + (1,e)")
#plt.savefig('JC_model.png')
plt.show()


#Iterate and verify model

n_list = np.arange(1,11,1)

theo_freq = g * np.sqrt(n_list)#theoretical values

freq_list = []



for n in n_list:
    psi = tensor(fock(n_cavity, n), basis(2,0)) + tensor(fock(n_cavity, n-1), basis(2,1))
    output = mesolve(H_theoretical, psi, tau, c_ops=[], e_ops=[a.dag() * a, sigma_plus * sigma_minus])
    #output_alt = mesolve(H_experimental, psi, times, c_ops=[], e_ops=[a.dag() * a, sigma_plus * sigma_minus])
    n_c = output.expect[0]
    n_a = output.expect[1]
    #n_a_alt = output_alt.expect[1]

    peaks, _ = find_peaks(n_a, height=0.5) 
    #peaks_alt, _ = find_peaks(n_a_alt, height=0.5)
    #fig, axes = plt.subplots(1, 1, figsize=(10, 6))
    #axes.plot(output.times, n_c, label="Cavity Photon Number")
    #axes.plot(output.times, n_a, label="Atom Excitation")
    #axes.scatter(tau[peaks], np.array(n_a)[peaks], color='red', s=10, marker='o', label='Peaks')  # Mark peaks
    #axes.grid(True)
    #axes.legend(loc='best')
    #axes.set_xlabel("Time (arb.units)")
    #axes.set_ylabel("Photon number")
    #axes.set_title(f"Photon and Atom Excitation Dynamics for n={n}")
    #plt.show()


    
    # Calculate time points of peaks
    peak_times = tau[peaks]
    #peak_times_alt = times[peaks_alt]
    periods = np.diff(peak_times)
    #print(f'{n}:',periods)
    #periods_alt = np.diff(peak_times_alt)
    average_period = np.mean(periods)
    #average_period_alt = np.mean(periods_alt) * wc
    #print(f'The theo period is {average_period} and the experimental period is {average_period_alt}')
    error_period = np.std(periods)
    frequency = 1 / average_period if average_period != 0 else 0

    freq_list.append(frequency * 2 * np.pi * wc)


plt.figure()
plt.scatter(n_list, freq_list, marker = 'x', label = 'experimental frequency')
plt.scatter(n_list, theo_freq, color = 'red', marker = 'x', label = 'theoretical prediction')
plt.xlabel('Fock state of cavity')
plt.ylabel('Frequency (arb. units)')
plt.xscale('log')  # Setting log scale for x-axis
plt.yscale('log')
plt.grid(True)
plt.legend(loc='best')
plt.savefig('Rabi_oscillations_2.png')
plt.show()

### Simple MA fix

In [ ]:
from scipy.ndimage import uniform_filter1d

Pe_smooth = uniform_filter1d(Pe, size=5)  # Smooth with a window of 5 points
peaks, _ = find_peaks(Pe_smooth, height=0.5)

In [70]:
#####

In [194]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from scipy.signal import find_peaks

# =============================================================================
# Define Experimental Parameters (in SI units)
# =============================================================================
wc = 2 * np.pi * 5e9   # cavity frequency (5 GHz)
wa = 2 * np.pi * 5e9   # atomic transition frequency (5 GHz)
g  = 2 * np.pi * 1e7   # coupling strength (10 MHz)

# This values were defined by ChatGPT based on similarity with superconducting qubits' values.
# wc and wa are set at resonance, we might change this, but I think Florian wanted this asw.

# =============================================================================
# Define Unitless (Dimensionless) Parameters
# =============================================================================
g_eff = g / wc       # unitless coupling
delta_eff = (wa - wc) / wc  # unitless detuning, this should be zero since wa == wc, set at resonance at the beginning -^

# =============================================================================
# Set Hilbert Space Dimensions and Define Operators
# =============================================================================
n_cavity = 75  # number of cavity Fock states to include, we might increase this but it's basically the same.

# Define operators on the full (cavity ⊗ atom) space.
a      = tensor(destroy(n_cavity), qeye(2))   # cavity annihilation operator
adag   = a.dag()                       # cavity creation operator
num_cav = adag * a                     # cavity photon number operator

# Two-level system (atom) operators (QuTiP’s sigmam() and sigmap() use basis |e>,|g> with sigmaz = diag(1, -1))
sm = tensor(qeye(n_cavity), sigmam())  # atomic lowering operator
sp = tensor(qeye(n_cavity), sigmap())  # atomic raising operator
sz = tensor(qeye(n_cavity), sigmaz())  # atomic inversion

# =============================================================================
# Build the Unitary (Time-Independent) Hamiltonian
# =============================================================================
H = num_cav + 0.5 * (wa / wc) * sz + g_eff * (a * sp + adag * sm)

# =============================================================================
# Time Evolution for a Single Initial State
# =============================================================================
# The new starting state is a superposition of two states:
# psi_0 = tensor(fock(n_cavity, 1), fock(2, 1)) + tensor(fock(n_cavity, 2), fock(2, 0))
psi0 = (tensor(fock(n_cavity, 2), basis(2, 0)) + tensor(fock(n_cavity, 2), basis(2, 0))).unit()

# Choose a time grid in the unitless time variable (wc*t).
t_max = 5000
n_steps = 5000
tlist = np.linspace(0, t_max, n_steps)  # Time grid

# Define the projection onto the excited state (atom) operator:
excited_proj = tensor(qeye(n_cavity), basis(2, 0) * basis(2, 0).dag())

# Solve the time-dependent Schrödinger equation (no collapse operators).
result = mesolve(H, psi0, tlist, [], [num_cav, excited_proj])

# Plot the cavity photon number and atomic excitation probability vs. time.
plt.figure(figsize=(8, 5))
plt.plot(tlist, result.expect[0], label="Cavity Photon Number")
plt.plot(tlist, result.expect[1], label="Atom Excitation Probability")
plt.xlabel("Dimensionless Time (ω_c t)")
plt.ylabel("Expectation Value")
plt.title("JC Dynamics: Cavity & Atom Observables")
plt.legend()
plt.tight_layout()
plt.show()

# =============================================================================
# Verifying the Simulation: Extracting the Rabi Frequencies
# =============================================================================
n_manifolds = np.arange(1, 11)  # simulate for n = 1,2,...,10 (total excitation numbers)
omega_exp_list = []  # will hold frequencies extracted from simulation (in units of ω_c)
omega_th_list = []   # theoretical frequencies

for n in n_manifolds:
    # Correct initial state: |n,g> + |n-1,e>
    psi0_n = tensor(fock(n_cavity, n), basis(2, 0)) + tensor(fock(n_cavity, n-1), basis(2, 1))
    
    # Normalize the state
    psi0_n = psi0_n.unit()

    # Theoretical Rabi frequency
    Omega_theory = 2 * g_eff * np.sqrt(n)
    T_R = 2 * np.pi / Omega_theory
    omega_th_list.append(Omega_theory)
    
    # Time grid for oscillations
    t_max_n = 3 * T_R
    n_steps_n = 1000
    tlist_n = np.linspace(0, t_max_n, n_steps_n)
    
    # Solve dynamics
    result_n = mesolve(H, psi0_n, tlist_n, [], [excited_proj])
    Pe = result_n.expect[0]  # atomic excitation probability
    
    # Extract frequency from oscillations
    peaks, _ = find_peaks(Pe, height=0.5 if n < 4 else 0.5)
    
    if len(peaks) >= 2:
        T_meas = np.mean(np.diff(tlist_n[peaks]))
        Omega_meas = 2 * np.pi / T_meas
    else:
        Omega_meas = np.nan  # not enough peaks detected
    
    omega_exp_list.append(Omega_meas)

omega_exp_list = np.array(omega_exp_list)
omega_th_list = np.array(omega_th_list)

# Scatter plot comparing simulated (experimental) vs. theoretical Rabi frequencies.
plt.figure(figsize=(6, 5))
plt.scatter(n_manifolds, omega_exp_list, color='b', label="Simulated Ω")
plt.plot(n_manifolds, omega_th_list, 'r--', label="Theoretical Ω = 2 g_eff √n")
plt.xlabel("Total Excitation Number n")
plt.ylabel("Rabi Frequency (in units of ω_c)")
plt.title("Comparison of Rabi Frequencies")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#####

In [143]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

# =============================================================================
# Define Parameters
# =============================================================================
wc = 2 * np.pi * 5e9   # Cavity frequency (5 GHz)
wa = 2 * np.pi * 5e9   # Atom frequency (5 GHz)
g  = 2 * np.pi * 1e7   # Coupling strength (10 MHz)

g_eff = g / wc       # Dimensionless coupling strength

# =============================================================================
# Define Operators
# =============================================================================
n_cavity = 15  # Increased number of cavity Fock states

a = tensor(destroy(n_cavity), qeye(2))  
adag = a.dag()   
num_cav = adag * a  

sm = tensor(qeye(n_cavity), sigmam())  
sp = tensor(qeye(n_cavity), sigmap())  
sz = tensor(qeye(n_cavity), sigmaz())  

# =============================================================================
# Corrected Jaynes-Cummings Hamiltonian
# =============================================================================
H_theoretical = num_cav + 0.5 * sz + g_eff * (a.dag() * sm + a * sp)

# =============================================================================
# Correct Initial State: |2,g⟩ + |1,e⟩
# =============================================================================
psi1 = (1/np.sqrt(2)) * tensor(fock(n_cavity, 2), basis(2, 0))
psi2 = (1/np.sqrt(2)) * tensor(fock(n_cavity, 1), basis(2, 1))

psi0 = psi1 + psi2

# =============================================================================
# Time Evolution Setup
# =============================================================================
T_R = 2 * np.pi / (2 * g_eff * np.sqrt(2))  # Rabi period for n=2
times = np.linspace(0, 10 * T_R, 5000)  # More oscillations, but fewer steps to reduce precision errors

# Solve the time evolution
result = mesolve(H_theoretical, psi0, times, c_ops=[], e_ops=[a.dag() * a, sp * sm])

# =============================================================================
# Plot Results
# =============================================================================
n_c = result.expect[0]  # Cavity photon number expectation
n_a = result.expect[1]  # Atomic excitation probability expectation

fig, axes = plt.subplots(1, 1, figsize=(10, 6))
axes.plot(result.times, n_c, label="Cavity Photon Number")
axes.plot(result.times, n_a, label="Atom Excitation Probability")
axes.grid(True)
axes.legend(loc='best')
axes.set_xlabel("Time (arb. units)")
axes.set_xlim(0, 10 * T_R)
axes.set_ylabel("Expectation Value")
axes.set_title("Corrected Jaynes-Cummings Dynamics: (2,g) + (1,e)")
plt.show()


In [151]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

# =============================================================================
# Define Parameters
# =============================================================================
wc = 2 * np.pi * 5e9   # Cavity frequency (5 GHz)
wa = 2 * np.pi * 5e9   # Atom frequency (5 GHz)
g  = 2 * np.pi * 1e7   # Coupling strength (10 MHz)

g_eff = g / wc       # Dimensionless coupling strength

# =============================================================================
# Define Operators
# =============================================================================
n_cavity = 15  # Increased number of cavity Fock states

a = tensor(destroy(n_cavity), qeye(2))  
adag = a.dag()   
num_cav = adag * a  

sm = tensor(qeye(n_cavity), sigmam())  
sp = tensor(qeye(n_cavity), sigmap())  
sz = tensor(qeye(n_cavity), sigmaz())  

# =============================================================================
# Corrected Jaynes-Cummings Hamiltonian
# =============================================================================
H_theoretical = num_cav + 0.5 * sz + g_eff * (a.dag() * sm + a * sp)

# =============================================================================
# Correct Initial State: |2,g⟩ + |1,e⟩
# =============================================================================
psi0 = (1/np.sqrt(2)) * (tensor(fock(n_cavity, 2), basis(2, 0)) + tensor(fock(n_cavity, 1), basis(2, 1)))

# =============================================================================
# Checking QuTiP Basis Ordering
# =============================================================================
sigma_z_atomic = sigmaz()  # Atomic-only operator
test_state = basis(2, 1)  # Expected to be |e⟩

if np.real(test_state.dag() * sigma_z_atomic * test_state) < 0:
    print("QuTiP uses inverted atomic basis! Using |g⟩ as the projection for 'excited' state.")
    excited_proj = tensor(qeye(n_cavity), basis(2, 0) * basis(2, 0).dag())  # If QuTiP flips |e⟩ and |g⟩
else:
    print("QuTiP uses standard atomic basis. Using |e⟩ as the projection.")
    excited_proj = tensor(qeye(n_cavity), basis(2, 1) * basis(2, 1).dag())  # Normal convention

# =============================================================================
# Time Evolution Setup
# =============================================================================
T_R = 2 * np.pi / (2 * g_eff * np.sqrt(2))  # Rabi period for n=2
times = np.linspace(0, 10 * T_R, 5000)  # More oscillations, but fewer steps to reduce precision errors

# Solve the time evolution
result = mesolve(H_theoretical, psi0, times, c_ops=[], e_ops=[num_cav, excited_proj, sz])

# =============================================================================
# Plot Results
# =============================================================================
n_c = result.expect[0]  # Cavity photon number expectation
n_a = result.expect[1]  # Atomic excitation probability expectation
sz_values = result.expect[2]  # Expectation value of σ_z to verify QuTiP's basis

fig, axes = plt.subplots(1, 1, figsize=(10, 6))
axes.plot(result.times, n_c, label="Cavity Photon Number")
axes.plot(result.times, n_a, label="Atom Excitation Probability")
axes.grid(True)
axes.legend(loc='best')
axes.set_xlabel("Time (arb. units)")
axes.set_xlim(0, 10 * T_R)
axes.set_ylabel("Expectation Value")
axes.set_title("Corrected Jaynes-Cummings Dynamics: (2,g) + (1,e)")
plt.show()

# =============================================================================
# Verify if QuTiP Basis is Flipped
# =============================================================================
plt.figure(figsize=(8, 4))
plt.plot(result.times, sz_values, label="⟨σ_z⟩")
plt.axhline(0, color="black", linestyle="--")
plt.xlabel("Time (arb. units)")
plt.ylabel("⟨σ_z⟩")
plt.title("Checking QuTiP Basis Ordering via ⟨σ_z⟩")
plt.legend()
plt.grid()
plt.show()

QuTiP uses inverted atomic basis! Using |g⟩ as the projection for 'excited' state.


In [ ]:
#

In [169]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from scipy.signal import find_peaks

# =============================================================================
# Define Experimental Parameters (in SI units)
# =============================================================================
wc = 2 * np.pi * 5e9   # cavity frequency (5 GHz)
wa = 2 * np.pi * 5e9   # atomic transition frequency (5 GHz)
g  = 2 * np.pi * 1e7   # coupling strength (10 MHz)

# =============================================================================
# Define Unitless (Dimensionless) Parameters
# =============================================================================
g_eff = g / wc       # unitless coupling
delta_eff = (wa - wc) / wc  # unitless detuning, this should be zero since wa == wc

# =============================================================================
# Set Hilbert Space Dimensions and Define Operators
# =============================================================================
n_cavity = 75  # number of cavity Fock states to include

# Define operators on the full (cavity ⊗ atom) space.
a      = tensor(destroy(n_cavity), qeye(2))   # cavity annihilation operator
adag   = a.dag()                       # cavity creation operator
num_cav = adag * a                     # cavity photon number operator

# Two-level system (atom) operators (QuTiP's basis is reversed: |0⟩ = excited, |1⟩ = ground)
sm = tensor(qeye(n_cavity), sigmam())  # atomic lowering operator
sp = tensor(qeye(n_cavity), sigmap())  # atomic raising operator
sz = tensor(qeye(n_cavity), sigmaz())  # atomic inversion

# =============================================================================
# Build the Unitary (Time-Independent) Hamiltonian
# =============================================================================
H = num_cav + 0.5 * (wa / wc) * sz + g_eff * (a * sp + adag * sm)

# =============================================================================
# Time Evolution for a Single Initial State
# =============================================================================
# The new starting state (corrected for QuTiP's inverted basis)
# QuTiP's |0> is actually the excited state, |1> is ground
psi0 = (tensor(fock(n_cavity, 2), basis(2, 0)) + tensor(fock(n_cavity, 2), basis(2, 0))).unit()

# Choose a time grid in the unitless time variable (wc*t).
t_max = 5000
n_steps = 5000
tlist = np.linspace(0, t_max, n_steps)  # Time grid

# Define the projection onto the excited state (corrected for QuTiP basis)
excited_proj = tensor(qeye(n_cavity), basis(2, 0) * basis(2, 0).dag())  # Tracks |e⟩ (QuTiP's basis(2,0))

# Solve the time-dependent Schrödinger equation (no collapse operators).
result = mesolve(H, psi0, tlist, [], [num_cav, excited_proj])

# Plot the cavity photon number and atomic excitation probability vs. time.
plt.figure(figsize=(8, 5))
plt.plot(tlist, result.expect[0], label="Cavity Photon Number")
plt.plot(tlist, result.expect[1], label="Atom Excitation Probability")
plt.xlabel("Dimensionless Time (ω_c t)")
plt.ylabel("Expectation Value")
plt.title("JC Dynamics: Cavity & Atom Observables")
plt.legend()
plt.tight_layout()
plt.show()

# =============================================================================
# Verifying the Simulation: Extracting the Rabi Frequencies
# =============================================================================
n_manifolds = np.arange(1, 11)  # simulate for n = 1,2,...,10 (total excitation numbers)
omega_exp_list = []  # will hold frequencies extracted from simulation (in units of ω_c)
omega_th_list = []   # theoretical frequencies

for n in n_manifolds:
    # Corrected initial state in QuTiP convention: |n,e> + |n-1,g>
    psi0_n = tensor(fock(n_cavity, n), basis(2, 0)) + tensor(fock(n_cavity, n-1), basis(2, 1))
    
    # Normalize the state
    psi0_n = psi0_n.unit()

    # Theoretical Rabi frequency
    Omega_theory = 2 * g_eff * np.sqrt(n)
    T_R = 2 * np.pi / Omega_theory
    omega_th_list.append(Omega_theory)
    
    # Time grid for oscillations
    t_max_n = 3 * T_R
    n_steps_n = 1000
    tlist_n = np.linspace(0, t_max_n, n_steps_n)
    
    # Solve dynamics
    result_n = mesolve(H, psi0_n, tlist_n, [], [excited_proj])
    Pe = result_n.expect[0]  # atomic excitation probability
    
    # Extract frequency from oscillations
    peaks, _ = find_peaks(Pe, height=0.5 if n < 4 else 0.5)
    
    if len(peaks) >= 2:
        T_meas = np.mean(np.diff(tlist_n[peaks]))
        Omega_meas = 2 * np.pi / T_meas
    else:
        Omega_meas = np.nan  # not enough peaks detected
    
    omega_exp_list.append(Omega_meas)

omega_exp_list = np.array(omega_exp_list)
omega_th_list = np.array(omega_th_list)

# Scatter plot comparing simulated (experimental) vs. theoretical Rabi frequencies.
plt.figure(figsize=(6, 5))
plt.scatter(n_manifolds, omega_exp_list, color='b', label="Simulated Ω")
plt.plot(n_manifolds, omega_th_list, 'r--', label="Theoretical Ω = 2 g_eff √n")
plt.xlabel("Total Excitation Number n")
plt.ylabel("Rabi Frequency (in units of ω_c)")
plt.title("Comparison of Rabi Frequencies")
plt.legend()
plt.tight_layout()
plt.show()


In [165]:
from qutip import *
import numpy as np

# Define atomic basis states in QuTiP
e_qutip = basis(2, 0)  # Expected to be |e⟩
g_qutip = basis(2, 1)  # Expected to be |g⟩

# Define sigma_z for atomic space
sigma_z_atomic = sigmaz()

# Compute expectation values
expect_e = np.real(e_qutip.dag() * sigma_z_atomic * e_qutip)
expect_g = np.real(g_qutip.dag() * sigma_z_atomic * g_qutip)

print("QuTiP Basis Definitions:")
print("e_qutip = basis(2,0):", e_qutip.full().flatten())
print("g_qutip = basis(2,1):", g_qutip.full().flatten())

print("\nEigenvalues of σ_z:")
print("⟨e|σ_z|e⟩ =", expect_e)
print("⟨g|σ_z|g⟩ =", expect_g)

# Verify tensor product structure
n_cavity = 5
psi_test = tensor(fock(n_cavity, 0), e_qutip)
expect_tensor = np.real(psi_test.dag() * tensor(qeye(n_cavity), sigma_z_atomic) * psi_test)

print("\nExpectation of σ_z in tensor space for (|0,e⟩):", expect_tensor)

QuTiP Basis Definitions:
e_qutip = basis(2,0): [1.+0.j 0.+0.j]
g_qutip = basis(2,1): [0.+0.j 1.+0.j]

Eigenvalues of σ_z:
⟨e|σ_z|e⟩ = 1.0
⟨g|σ_z|g⟩ = -1.0

Expectation of σ_z in tensor space for (|0,e⟩): 1.0
